<a href="https://colab.research.google.com/github/ssanchezgoe/iue_curso_ia/blob/main/nb_google_colab/s04_Pandas_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  <tr>
     <th><p><img alt="Colaboratory logo" height="120 px" src="http://www.redttu.edu.co/es/wp-content/uploads/2016/01/iue.png" align="left" hspace="10px" vspace="0px"></p></th> 
     <th><h1>  Python: Pandas II </h1></th>
  </tr>

<p><a name="contents"></a></p>

# Contenido Sesión 4

- <a href="#misdat">1. Procesamiento de datos nulos II</a><br>
 - <a href="#ignorarFila">1.1  Ignorar fila de datos</a><br>
 - <a href="#valVec">1.2 Llenado con valores vecinos</a><br>
 - <a href="#valSKL">1.3 Llenado de datos usando sklearn</a><br>
 - <a href="#valMod">1.4 Llenado con modelado</a><br>
- <a href="#example"> Ejemplo de preprocesado de datos</a><br>
- <a href="#catvar">2. Manipulación de variables categóricas</a><br>
 - <a href="#opevec">2.1. Operaciones vectorizadas con `strings`</a><br>
 - <a href="#varord">2.2. Variables ordinales: Codificación de enteros</a><br>
 - <a href="#varnom">2.3. Variables nominales: Condificación one-hot (dummy variables)</a><br>
- <a href="#groupby">3. Groupby </a><br>
 - <a href="#trafil">3.1. Funciones ` transform`  y ` filter` </a><br>
 - <a href="#cladiv">3.2. Especificando la clave para la división del DataFrame  </a><br>
 - <a href="#tabdin">3.3. Tablas dinámicas  </a><br>




<p><a name="misdat"></a></p>

# 1. Procesamiento de datos nulos II

[[Contenidos]](#contents)

El dataset adult.csv fue extraído por Barry Becke de la base de datos de Censos de  1994. Contiene tanto variables numéricas como categóricas. La información de las variables y su contenido se presentan a continuación:

age: continuo \\
workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. \\
fnlwgt: continuo. \\
education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. \\
education-num: continuo. \\
marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. \\
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. \\
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. \\
race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. \\
sex: Female, Male. \\
capital-gain: continuo. \\
capital-loss: continuo. \\
hours-per-week: continuo. \\
native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [ ]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/diplomado-bigdata-machinelearning-udea/Curso1/master/s04/adult.csv') 

In [ ]:
df.head()

Encontrar el número de valores faltantes por columna

In [ ]:
df.isnull().sum()

In [ ]:
# Porcentaje de información faltante
df.isnull().sum()/len(df)*100

Podemos decir que hay cinco metodologías principales para hacer el tratamiento de los datos nulos:
  
1.  Reemplazar con un valor constante
2. Reemplazar con un valor estadístico  (promedio, media, moda )
3. Ignorar la fila de datos
4.  Hacer un llenado hacia atrás (back-fill) o hacia adelante (forward-fill)
5. Llenado basado en modelos

Los dos primeros ya los cubrimos en la sesión anterior, así que enfoquemonos en los restantes.



<p><a name="ignorarFila"></a></p>

## 1.1 Ignorar fila de datos

El comando `dropna()` permite eliminar las filas y/o columnas en las que hayan datos faltantes

In [ ]:
df_filtered=df.dropna()

In [ ]:
print('Número de filas iniciales', len(df))
print('Número de filas después de filtrar', len(df_filtered))
print('Porcentaje de filas eliminadas',(1-len(df_filtered)/len(df))*100)

In [ ]:
df_filtered.iloc[10:20,:]

Note que los índices no cambian. Lo que realiza es la eliminación de la fila (por ejemplo la 14 no está), pero mantiene la indexación. Por tanto, estos no coinciden con el número total de filas. Para reasignar los índices se puede hacer uso del comando `reset_index()`  

In [ ]:
df_filtered.reset_index(drop=True,inplace=True)

In [ ]:
df_filtered.tail()

Se puede usar el argumento `subset` para seleccionar solo las columnas sobre las que se desea analizar si hay valores nulos para eliminar las filas 

In [ ]:
df.dropna(subset=['native-country'])

<p><a name="valVec"></a></p>

## 1.2 Llenado con valores vecinos

El comando `fillna()` tiene el argumento `method` que permite hacer un llenado hacia atrás ('bfill') o hacia adelante ('ffill').

Tenga en cuenta que al usar ésta opción si el dato con el que se intenta hacer el llenado también es un NaN, éste permanece.

In [ ]:
df.iloc[25:30,:]

In [ ]:
df.iloc[25:30,:].fillna(method='bfill')

In [ ]:
df.iloc[25:30,:].fillna(method='ffill')

<p><a name="valSKL"></a></p>

## 1.3 Llenado de datos usando sklearn



Es posible también usar el metodo 'SimpleImputer' de la libreria sklearn para hacer llenado, éste nos permite definir cual estrategia usar para el llenado de los datos (media, mediana o moda).

In [ ]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy='most_frequent')
X = df.iloc[25:30,:]
trans = pd.DataFrame(imp.fit_transform(X))
trans



<p><a name="valMod"></a></p>

## 1.4 Llenado con modelado 

Una forma muy eficiente, pero más compleja de llenado de datos es convertir la columna a la que deseamos llenarle los datos faltantes y modelarla en función de las columnas sin datos faltantes.

Dependiendo de la clase de dato podemos usar regresiones o clasificaciones. No profundizaremos en ésta estrategia pues aún no conocemos algoritmos de ML para ello.

<p><a name="example"></a></p>

# Ejemplo de preprocesado de datos

[[Contenidos]](#contents)

Trataremos un ejemplo de una base de datos de una tabla de características de marcas de carros.

De esta base de datos, pretendemos resolver el siguiente problema


**Cual es la tasa de consumo en Litros por cada 100 km para un carro diesel?**

## Importe de datos

Los datos en crudo se pueden encontrar en la dirrección  https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data.

**Importar panda y matplotlib.pyplot:**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

### Lectura de los datos desde un url

In [ ]:
archivo = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/auto.csv"

### Cabecera

In [ ]:
headers = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]

### Carga de datos desde el url

Para esto usamos el método `read_csv()`

In [ ]:
df = pd.read_csv(archivo, names = headers)

### Inspección del archivo

Recordemos que podemos usar el método head()

In [ ]:
df.head()

Podemos observar que muchas de las entradas están etiquetadas con el símbolo `?`. Esto corresponde datos faltantes que pueden dificultar un análisis ulterior.

Los pasos que debemos seguir a continuación son:



1.   Identificar los valores faltantes
2.   Tratar los valores faltantes.
3.   Corregir el formato de los datos.



## Identificación de valores faltantes y tratamiento.


### Conversión de `?` a `NaN`

Quienes llenaron esta tabla, identificaron los valores faltantes como `?`.  Por razones de tiempo de computo y conveniencia, replazaremos estos signos de interrogación por `NaN`, para lo cual debemos importantar antes la librería numéricas de python numpy. 

Para realizar el reemplazo usaremos el método `replace()`.

In [ ]:
import numpy as np

df.replace("?", np.nan, inplace = True) # No olvidarse de usar inplace.
df.head(5)

### Identificación y conteo de valores faltantes:

Apliquemos el método `isnull()` y utilicemos la función de agregación `sum()`

In [ ]:
datos_faltantes = df.isnull().sum()
datos_faltantes

De acuerdo a la información anterior tenemos los siguientes datos faltantes.
<ol>
    <li>"normalized-losses": 41 </li>
    <li>"num-of-doors": 2 </li>
    <li>"bore": 4 </li>
    <li>"stroke" : 4 </li>
    <li>"horsepower": 2 </li>
    <li>"peak-rpm": 2 </li>
    <li>"price": 4 </li>
</ol>

## Tratamiento de los datos faltantes

<ol>
    <li>Eliminar el dato entero<br>
        a. Eliminar la fila entera<br>
        b. Eliminar la columna entera
    </li>
    <li>Reemplazar el dato<br>
        a. Reemplazar por la media<br>
        b. Reemplazar por la moda<br>
        c. Reemplazar basados en otra función
    </li>
</ol>

Solo debemos eliminar una columna si la mayoria de entradas son vacias. En nuestro caso, ninguna de las columnas cumplen con este criterio como para ser eliminadas. 

En este caso tenemos cierta libertad para aplicar diferences métodos de llenado de datos faltantes; no obstante, algunos métodos pueden resultar más adecuados que otros. Aplicaremos los siguientes métodos para cada columna:

<b>Reemplazo por la media:</b>
<ul>
    <li>"normalized-losses": 41 datos faltantes</li>
    <li>"stroke": 4 datos faltantes</li>
    <li>"bore": 4 datos faltantes</li>
    <li>"horsepower": 2 datos faltantes</li>
    <li>"peak-rpm": 2 datos faltantes</li>
</ul>

<b>Replace por la moda:</b>
<ul>
    <li>"num-of-doors": 2 dátos faltantes que serán reemplazados por "four". 
        <ul>
            <li>Razón: 84% de los sedan son 4 puertas. Dado que los carros cuatro puertas son los más frecuentes, la probabilidad de ocurrencia es mayor.</li>
        </ul>
    </li>
</ul>

<b>Eliminación de una fila entera:</b>
<ul>
    <li>"price": 4 datos faltantes
        <ul>
            <li>Razón: Como, eventualmente, el precio es algo que se puede predecir, ninguna entrada sin precio puede ser usada para una predicción; por ende, cualquier fila sin precio no será útil.</li>
        </ul>
    </li>
</ul>

### Cálculo del valor medio

In [ ]:
avg_norm_loss = df["normalized-losses"].astype("float").mean(axis=0)
print("Promedio de normalized-losses:", avg_norm_loss)

Reemplazo de  "NaN" por el promedio en la columna "normalized-losses":

In [ ]:
df["normalized-losses"].replace(np.nan, avg_norm_loss, inplace=True)

Calculo del valor medio para la columna 'bore':

In [ ]:
avg_bore=df['bore'].astype('float').mean(axis=0)
print("Average of bore:", avg_bore)

Reemplazo de `NaN` por  el valor medio

In [ ]:
df["bore"].replace(np.nan, avg_bore, inplace=True)

**Ejercicio: De acuerdo a lo anterior reemplace los valores faltantes de la columna stroke por su valor medio**

De doble click <b>aquí</b> para ver la solución:

<!-- The answer is below:

# Cálculo del valor medio:
avg_stroke = df["stroke"].astype("float").mean(axis = 0)
print("Average of stroke:", avg_stroke)

# Reemplazo de los valores faltantes por el valor medio
df["stroke"].replace(np.nan, avg_stroke, inplace = True)

-->

Cálculo del valor medio de la columna 'horsepower' y reemplazo en los valores faltantes:

In [ ]:
avg_horsepower = df['horsepower'].astype('float').mean(axis=0)
print("Average horsepower:", avg_horsepower)
df['horsepower'].replace(np.nan, avg_horsepower, inplace=True)

Columna de 'peak-rpm':

In [ ]:
avg_peakrpm=df['peak-rpm'].astype('float').mean(axis=0)
print("Average peak rpm:", avg_peakrpm)
df['peak-rpm'].replace(np.nan, avg_peakrpm, inplace=True)

Para ver cuáles valores están presentes en una columna particular, podemos usar el método `value_counts()`:

In [ ]:
df['num-of-doors'].value_counts()

Vemos que los carros de cuatro puertas son los más comunes. También podemos usar el método `.idxmax()` para calcular el tipo más común automaticamente:

In [ ]:
df['num-of-doors'].value_counts().idxmax()

Reemplazamos en la columna 'num-of-doors' los valores vacios por la moda:

In [ ]:
df["num-of-doors"].replace(np.nan, "four", inplace=True)

Finalmente, eliminemos todas las filas de carros sin precios:

In [ ]:
# Eliminación de columnas sin precio mediante dropna
df.dropna(subset=["price"], axis=0, inplace=True)

# Reinicio de los indices
df.reset_index(drop=True, inplace=True)

Apariencia de la tabla luego de preprocesarla

In [ ]:
df.head()

## Corrección de datos

<p>El último paso en el preprocesado de los datos consiste en revisar de que todos los datos estén en el formato adecuado(int, float, text u otro).</p>

Podemos usar en Pandas
<p><b>.dtypes()</b>  para ver el tipo</p>
<p><b>.astype()</b> para cambiar el tipo</p>

### Lista de tipos

In [ ]:
df.dtypes

<p> Se puede observa que algunas columnas no tienen el tipo correcto.
  
  Las variables numéricas deben ser de tipo 'float' o 'int', y las variables con caracteres, como categorias, deben ser del tipo 'object'. Por ejemplo, 'bore' (diámetro del cilindro) y 'stroke' (ciclos/tiempos) son variables numéricas que describen el motor, esperamos entonces que sean de tipo 'float' o 'int'; no obstante, son de tipo 'object'. Debemos convertirlas a un tipo adecuado usantdo el método "astype()".</p> 

### Conversión al tipo adecuado:

In [ ]:
df[["bore", "stroke"]] = df[["bore", "stroke"]].astype("float")
df[["normalized-losses"]] = df[["normalized-losses"]].astype("int")
df[["price"]] = df[["price"]].astype("float")
df[["peak-rpm"]] = df[["peak-rpm"]].astype("float")

In [ ]:
df.dtypes

Hemos obtenido, finalmente, un conjunto de datos correcto, sin datos faltantes y en un formato correcto.

## Estandarización de datos

<p>
Los datos se recolectan, normalmente, de diferentes concesionarios, con diferentes formatos.
(La estandarización de datos se refiere también a un tipo particular de normalización, en donde se extrae el promedio y se divíde por la desviación estándar.)
</p>
    
<b>¿Qué es la estandarización?</b>
<p>La estandarización es el proceso de transformar unos datos en un formato común que permita a los investigadores realizar comparaciones significativas.
</p>

<b>Ejemplo</b>
<p>Transformar mpg (millas por galon) a L/100km (Litros por cada 100 km):</p>
<p>En el conjunto de datos, "city-mpg" y "highway-mpg" están representadas en unidades de mpg. En europa, por ejemplo, el consumo se da en L/100km </p>
<p>Debemos aplicar una <b>transformación de los datos</b> para pasar de  mpg a L/100km?</p>

<p>La conversión es<p>
L/100km = 235 / mpg
<p>Podemos aplicar en Pandas operaciones matemáticas directamente para lograr este fin.</p>

In [ ]:
df.head()

In [ ]:
# Conversión de mpg a L/100km
df['city-L/100km'] = 235/df["city-mpg"]

# Visualización de la operación
df.head()

**Ejercicio:** De acuerdo con el procedimiento anterior, transforme mpg a L/100km en la columna de "highway-mpg", y cambie el nombre de la columna a highway-L/100km".

De doble click <b>aquí</b> para ver la solución.

<!-- 

# mpg a L/100km
df["highway-mpg"] = 235/df["highway-mpg"]

# renombre de "highway-mpg" a "highway-L/100km"
df.rename(columns={'highway-mpg':'highway-L/100km'}, inplace=True)

# visualización del conjunto de datos
df.head()

-->

## Normalización de los datos

<b>¿Por qué debemos aplica normalización?</b>
<p>La normalización es el proceso de transforma valores de varias variables a un rango similar. Normalmente, la normalización incluye un escalado de las varialbres de tal forma que el promedio sea 0 y la varianza 1, o escalar las variables de tal forma que el rango esté definido entre 0 y 1.
</p>

<b>Ejemplo</b>
<p>Supongamos que queremos rescalar las columnas "length", "width" and "height" </p>
<p><b>Objetivo:</b> Normalizar las variables a un rango de 0 a 1.</p>
<p><b>Métodología:</b> reemplazar el valor por (value original)/(valor máximo)</p>

In [ ]:
df['length'] = df['length']/df['length'].max()
df['width'] = df['width']/df['width'].max()

**Ejercicio:** Normalice la columna "heigh".

De doble click <b>aquí</b> para ver la solución.

<!--

df['height'] = df['height']/df['height'].max() 
# Mostrar las columnas normalizadas.
df[["length","width","height"]].head()

-->

Podemos observar como el rango queda definido entre 0 y 1

## Bineado de los datos

<b>Por qué binear?</b>
<p>
    Binear es el proceso que consiste en transformar una variable numérica continua en bines categoricos discretos, con el fin de realizar un analisis por grupos.
</p>

<b>Ejemplo: </b>
<p>En nuestra base de datos, "horsepower" es una variable real que varia desde 48 a 288, tiene 57 valores únicos. Que pasaría si quisieramos discriminar entre diferentes precios dependiendo de si se trata de un "horsepower" alto, medio o bajo? Podemos reorganizarlos en bines para un analisis más simple? </p>

<p>Usemos el método de pandas `cut` para partiro en tres bines: </p>

<p>Primero convirtamos "horsepower al formato correcto:" </p>

In [ ]:
df.dtypes

In [ ]:
df["horsepower"]=df["horsepower"].astype(int, copy=True)

Realicemos un histograma de esta variable para ver como están distribuida:

In [ ]:
import matplotlib as plt
df.horsepower.hist()
plt.pyplot.xlabel("horsepower")
plt.pyplot.ylabel("count")
plt.pyplot.title("horsepower bins")

<p> Para establecer 3 bines de ancho igual, usamos la función de numpy <code>linspace(start_value, end_value, numbers_generated)</code> .</p>
<p>En estos casos tendríamos que</p>
 <p>start_value=min(df["horsepower"]).</p>
<p>end_value=max(df["horsepower"]).</p>
<p>Como queremos crea  3 bines del mismo ancho, usamos 4 como divisor /numbers_generated=4)</p>

In [ ]:
bins = np.linspace(min(df["horsepower"]), max(df["horsepower"]), 4)
bins

Nombre de los grupos

In [ ]:
group_names = ['Low', 'Medium', 'High']

Aplicaciónde la función "cut"  para determinar que valor de "df['horsepower']" pertenece a qué categoría. 

In [ ]:
df['horsepower-binned'] = pd.cut(df['horsepower'], bins, labels=group_names, include_lowest=True )
df[['horsepower','horsepower-binned']].head(20)

Veamos las cuentas por cada bin:

In [ ]:
df["horsepower-binned"].value_counts()

Grafiquemos como están distribuidos por estas categorias la columna "horsepower"

In [ ]:
import matplotlib as plt
df["horsepower-binned"].value_counts().plot(kind='bar')
# set x/y labels and plot title
plt.pyplot.xlabel("horsepower")
plt.pyplot.ylabel("count")
plt.pyplot.title("horsepower bins")

## Visualización de los bins

Normalmente,  un histograma se crea para ver la distribución de los bines que creamos arriba:

In [ ]:
import matplotlib as plt
from matplotlib import pyplot

a = (0,1,2)

# draw historgram of attribute "horsepower" with bins = 3
plt.pyplot.hist(df["horsepower"], bins = 3)

# set x/y labels and plot title
plt.pyplot.xlabel("horsepower")
plt.pyplot.ylabel("count")
plt.pyplot.title("horsepower bins")

En la gráfica anterior se muestra el resultado del bineado para el atributo "horsepower"

<p><a name="catvar"></a></p>

# 2. Manipulación de variables categóricas

[[Contenidos]](#contents)

Las variables categóricas son aquellas que contienen etiquetas en vez de valores numéricos. Exiten variables categóricas ordinales y nominales, las primeras son aquellas que tienen algún tipo de jeraquización, como por ejemplo el nivel de escolaridad:*primaria, secundaria, universitaria*; las segunas son aquellas que no se pueden ordenar como profesión: *abogado, médico, ingeniero* . Según el problema que se desea resolver y el tipo de variables categóricas, estás deben tener un tratamiento diferente.

Para ver más tipos de codificación ver: https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159

<p><a name="opevec"></a></p>

## 2.1. Operaciones vectorizadas con `strings`

[[Contenidos]](#contents)

Python ofrece una relativa facilidad en el manejo y manipulación de datos de tipo `str`. Pandas se basa en esto y proporciona un conjunto integral de operaciones vectorizadas

NumPy no proporciona un acceso tan simple. Por ejemplo, para escribir con mayúscula la primera letra de las entradas de un arreglo podemos escribir:



In [ ]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']       
[s.capitalize() for s in data]

Quizás esto sea suficiente para trabajar con algunos datos, pero no funcionará si faltan valores:

In [ ]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']       
[s.capitalize() for s in data]

Pandas incluye opciones para abordar tanto el manejo de operaciones vectorizadas como el manejo correcto de los datos faltantes a través del atributo `str` de los objetos `series` e `index` de Pandas

In [ ]:
names = pd.Series(data)    
names

Realicemos la operación anterior mediante el atributo `str` (note que se omiten los valores nulos)


In [ ]:
names.str.capitalize()

Casi todos los métodos incorporados de Python para el manejo de cadenas se reflejan en un método de vectorizado de cadenas de Pandas. Algunos de estos métodos son

*   `len()`
*   `lower()`
* `upper()`
*  `startswith()`
* `contains()`
*  `split()`
* `get()`
* `strip()`
* `replace()`



In [ ]:
monte = pd.Series([' Graham Chapman ', ' John Cleese', 'Terry Gilliam ', 'Eric Idle', 'Terry Jones', 'Michael Palin'])
monte

In [ ]:
# entradas en letra minúscula (str)
monte.str.lower() 

In [ ]:
# si la entrada contiene la letra T (bol)
monte.str.contains('T')

In [ ]:
# separar por palabra (list)
monte.str.split()

In [ ]:
# obtener el apellido de cada entrada
monte.str.split().str.get(-1)

Para una guia completa visitar: https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html

<p><a name="varord"></a></p>

## 2.2. Variables ordinales: Codificación de enteros

[[Contenidos]](#contents)

La codificación entera (Integer Encoding) consiste en asignar un valor entero a cada uno de los valores ordenados, según correponda.  

In [ ]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/diplomado-bigdata-machinelearning-udea/Curso1/master/s04/adult.csv') 
df.head()

Este tipo de implementación ya está hecha en el dataset para el caso del nivel educativo en las columnas education y education-num

In [ ]:
df[['education','education-num']].drop_duplicates().sort_values(by='education-num')

Con la función map es fácil de implementar en python. Por ejemplo para la columa class índiquemos con $1$ los $\leq 50$ y con $2$ los $>50$

In [ ]:
df['class'].unique()

In [ ]:
df['class-num']=df['class'].map({'<=50K':1, '>50K':2})

In [ ]:
df[['class-num','class']].head()

Podemos también convertir una columna a *category* y luego utilizar esos valores categóricos para la codificación entera. Por ejemplo, realicemos este proceso para la columna *workclass*. Convirtamos la columna al tipo *category*:

In [ ]:
df["workclass"] = df["workclass"].astype("category")
df.dtypes

Luego podemos asignar la variable codificada a una nueva columna utilizando `cat.codes`


In [ ]:
df["workclass_code"] = df["workclass"].cat.codes

In [ ]:
df.head()

In [ ]:
df[['workclass','workclass_code']].head()

<p><a name="varnom"></a></p>

## 2.3. Variables nominales: Condificación one-hot (dummy variables)

[[Contenidos]](#contents)

La codificación one-hot consiste asignar valores binarios $[1, 0]$ a las variables categóricas, en las que cada valor de la variable se convierte en un campo nuevo y se asigna el valor $[1, 0]$ si el evento contiene o no la variable según sea el caso.


In [ ]:
df.sex.unique()

Pandas viene con el comando `pd.get_dummies()` que facilita el trabajo y realiza la codeificación one-hot de forma muy sencilla. Note que al implementar este comando, el campo `sex` se elimina y solo quedan las variables dummy

In [ ]:
pd.get_dummies(df.sex).head()

Se puede implementar para el dataset completo y el comando `pd.get_dummies()` solo aplica sobre las variables categóricas.

In [ ]:
pd.get_dummies(df).head()

Aplicar este tipo de codificación puede aumentar significativamente el número de variables. Esto depende de la cantidad de posibles valores que tengan las variables categóricas

In [ ]:
len(df.columns)

In [ ]:
len(pd.get_dummies(df).columns)

<p><a name="groupby"></a></p>

# 3. Groupby

[[Contenidos]](#contents)

`Seaborn` es una libreria para hacer gráficos estadísticos en Python. Está construido sobre matplotlib y está estrechamente integrado con las estructuras de datos de pandas. Tiene incorporadas algunas bases de datos: 

https://github.com/mwaskom/seaborn-data

<p><a name="trafil"></a></p>

## 3.1. Funciones Transform y Filter

[[Contenidos]](#contents)

El dataset Planets, disponible a través del paquete Seaborn, da información sobre planetas descubiertos alrededor de otras estrellas. Se puede descargar via Seaborn de la siguiente manera. 

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd

planets = sns.load_dataset('planets')
planets.head()

Recordemos que Groupby nos ofrece las funciones `aggregate()` y `apply()`. 

`aggregate()` puede tomar una cadena, una función o una lista de estas, y calcular todos los agregados a la vez, mientras que `apply()` permite aplicar una función arbitraria a los resultados del agrupamiento

In [ ]:
planets.groupby("method").distance.aggregate(["min", "max", "median"])
#planets.groupby("method").distance.apply(np.min)

Si bien `aggregate()` debe devolver una versión reducida de los datos, `transform` puede devolver una versión transformada de los datos completos para que se recombinen. Para tal transformación, la salida tiene la misma forma que la entrada

In [ ]:
df2 = planets.groupby('method')[["distance"]].transform("median").drop_duplicates()
df2.reset_index(drop=True)


La operación `filter` permite eliminar datos en función de las propiedades del grupo. Note que esta rutina no filtra un marco de datos en su contenido. El filtro se aplica a las etiquetas del índice.

In [ ]:
# agrupemos por método y filtremos los que operan a una distancia media mayor a 500

def filter_func(x):
    return x["distance"].mean() > 500 #

planets.groupby('method').filter(filter_func).method.value_counts()

Ejemplo: Muestre el número de planetas descubiertos por método de detección y por década

In [ ]:
decade = 10 * (planets['year'] // 10)    # serie que contiene las decadas
decade = decade.astype(str) + 's'               
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

<p><a name="cladiv"></a></p>

## 3.2. Especificando la clave para la división del DataFrame 

[[Contenidos]](#contents)

Los ejemplos presentados anteriormente expresan solo unas de las muchas opciones mediante las cuales se pueden definir los grupos. Veamos algunas otras opciones para la especificación de grupos

In [ ]:
# la clave puede ser cualquier serie o lista con una longitud que coincida con la del DataFrame

rng = np.random.RandomState(0)        
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],'data1': range(1,7),'data2': rng.randint(0, 10, 6)}, 
                  columns = ['key', 'data1', 'data2']) 
print(df)

# agrupar las filas 0 y 2 con índice "a"; las 1 y 3 con índice 1 ; las 4 y 5 con índice 10 y obtener la media
L = ["a", 1, "a", 1, 10, 10]    
df.groupby(L).mean()

Otro método es el de proporcionar un diccionario que asigne los valores de los índices a las claves de grupo

In [ ]:
# asignar la columna "key" como índice
df2 = df.set_index('key')   
print(df2) 

# mapear los índices
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'} 
print(df2.groupby(mapping).sum())

Análogamente al mapeo, es posible pasar cualquier función de Python que ingrese el valor del índice y genere el grupo

In [ ]:
df2.groupby(str.lower).mean()

cualquiera de las opciones anteriores se pueden combinar para agrupar con índice múltiple

In [ ]:
mi = df2.groupby([str.lower, mapping]).mean()
mi

Se puede acceder a los diferentes niveles del índice múltiple mediante el argumento "level"

In [ ]:
mi.groupby(level=0).mean()

In [ ]:
mi.groupby(level=1).mean()

<p><a name="tabdin"></a></p>

## 3.3. Tablas dinámicas

[[Contenidos]](#contents)

La tabla dinámica toma los datos de las columnas como entrada y las agrupa en una tabla que proporciona un resumen multidimensional de los datos. Es esencialmente una versión multidimensional de la agregación con GroupBy.

En este caso importaremos el dataset "titanic" via seaborn, el cual contiene información sobre cada pasajero, incluyendo género, edad, clase, tarifa pagada y mucho más

In [ ]:
titanic = sns.load_dataset('titanic')
titanic.head()

Para comenzar a aprender más sobre este dataset, podríamos comenzar por agrupar según el género, el estado de supervivencia o alguna combinación de estos. Veamos la tasa de supervivencia por género:

In [ ]:
titanic.groupby('sex')[['survived']].mean()

Nos gustaría ir un paso más allá y ver la supervivencia por sexo y, por ejemplo, clase. Con GroupBy

In [ ]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

Este GroupBy bidimensional es lo suficientemente común como para que Pandas incluya una rutina conveniente pivot_table que maneja este tipo de agregación multidimensional

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class')

Por defecto `pivot_table` aplica la función `mean()`. Para cambiar la función de agregación utilizamos el argumento `aggfunc`

In [ ]:
titanic.pivot_table("survived", index='sex', columns='class', aggfunc="sum")

# agrupar por dos columnas y aplicar una función correspondiente a cada una

#titanic.pivot_table(index='sex', columns='class', aggfunc={'survived':sum, 'fare':'mean'})

la agrupación en tablas dinámicas se puede especificar con múltiples niveles. Podríamos estar interesados en ver la edad como una tercera dimensión. Seccionaremos la edad usando la función `pd.cut`

In [ ]:
age = pd.cut(titanic['age'], [0, 18, 80])       
titanic.pivot_table('survived', ['sex', age], 'class')

Podemos aplicar esta misma estrategia para trabajar con las columnas. Agreguemos información sobre la tarifa pagada usando `pd.qcut` para calcular automáticamente los cuantiles

In [ ]:
fare = pd.qcut(titanic['fare'], 2)       
multi = titanic.pivot_table('survived', ['sex', age], [fare, 'class'])
multi

El resultado es una agregación de cuatro dimensiones con índices jerárquicos.

In [ ]:
multi.mean(level=0)  

con la palabra clave `axis` podemos acceder a los niveles en las columnas

In [ ]:
multi.mean(axis=1,level="class")

A veces es útil calcular totales a lo largo de cada grupo. Esto se puede hacer a través de la palabra clave de `margin`

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True)

##Taller

A continuación cargaremos dos bases de datos con información referente a tarjetas de credito del mes 09 del 2017.

El archivo ESTADO_201709.txt tal que:

* ID:​ Nro. de identificación.
* Franquicia:​ A, B, C.
* Tipo Tarjeta:​ Representa el tipo de tarjeta.
* Número Tarjeta:​ Indica el número de la TC.
* Cupo Global:​ Indica el cupo de crédito que tiene la tarjeta.
* Disponible: Indica del cupo global cuanto tiene a ese corte disponible para realizar compras o avances.
* Fecha Emisión:​ Fecha en la que se compró la tarjeta.
* Fecha Ult Aumento Cupo: Fecha en la que se realizó el último aumento de cupo. Cuando está en 0 significa que nunca se ha realizado un aumento de cupo.
* Total Mes en Mora: Número de meses que ha estado en mora la TC (durante toda
la vida de la tarjeta).
*Altura de Mora: ​Indica la mora actual que tiene la TC. Viene en rango:
  ○ 0 No está en mora,
  ○ 30 está en mora hace menos de 30 días,
  ○ 60 está en mora hace menos de 60 días.
  ○ 90 está en mora hace menos de 90 días.
*Código Vendedor:​ Indica el código del vendedor que emitió la TC.
*Código Estado Tarjeta: Indica si la tarjeta está Activa o vigente (Codigo=0) y código = 1 (indica que la tarjeta está cancelada por el cliente).

Y el archivo TRANSACCION_201709.txt, tal que:

* ID:​ Nro de identificación.
* Número Tarjeta:​ Indica el número de la TC que realizó la transacción.
* Fecha de transacción​: Indica la fecha en que se realizó la transacción.
* Codigo Transacción:
  ○ AV : AVANCE
  ○ AB: ABONO
  ○ CN: COMPRA NACIONAL
  ○ CI: COMPRA INTERNAL
  ○ CM: CUOTA DE MANEJO.
* Valor Transacción:​ Indica el valor de la transacción.
* Numero Cuotas Diferidas: Indica el número de cuotas en el que el cliente desea pagar la compra.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

path = 'https://raw.githubusercontent.com/diplomado-bigdata-machinelearning-udea/Curso1/master/s04/'
dfE = pd.read_csv(path+'ESTADO_201709.txt', delimiter="|")
dfT = pd.read_csv(path+'TRANSACCION_201709.txt', delimiter="|")

### Responda
1. Imprima cada uno de los datasets para ver sus columnas
2. Use el método merge para combinar los datasets  en el `NUMERO TARJETA' y haciendo la intersección.
3. Busque los datos faltantes y determine una estrategia adecuada par su llenado.
4. Convierta todas las fechas al fomato adecuado
5. Genere un histograma de cada columna para ver su distribución
6. Agrupe los datos por tipo de tarjeta y estado de la tarjeta. Hága un conteo ¿Existe alguna relación entre el tipo de tarjeta y la cancelación de la misma?
7. Realice un bineo del total de meses en mora partiendolos en  tres categorías: 'baja mora','media mora','alta mora'. ¿Existe alguna relación entre la mora y la cancelación de la tarjeta?.
8. Haga un gráfico de dispersión 'CODIGO VENDEDOR' vs 'CODIGO ESTADO TARJETA', ¿El vendedor de la tarjeta influencia las cancelaciones?.

Recuerde hacer un analisis exploratorio de sus datos para familiarizarse con los datos antes de comenzar el análisis.